In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [34]:
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [36]:
ped=pd.read_csv("F:\\Python\\pedigree.csv",sep=",",encoding='gbk',low_memory=False)

In [37]:
ped.head()

,牛号,出生日期,父国别,父号,母国别,母号
0,110001000001,2000/1/6,CHN,11193546,CHN,110001099169
1,110001000003,2000/1/10,CHN,31195515,CHN,110001095046
2,110001000006,2000/1/16,CHN,31195515,CHN,110001094056
3,110001000007,2000/1/21,CHN,11194143,CHN,110001096113
4,110001000008,2000/1/20,CHN,11194143,CHN,110001090136


In [38]:
ped1=ped[['牛号','出生日期','父号','母号']]

# 所有牛号重新编号

In [39]:
a=pd.DataFrame()
b=pd.DataFrame()
c=pd.DataFrame()

In [40]:
a['牛号']=ped1['牛号']
b['牛号']=ped1['母号']
c['牛号']=ped1['父号']

In [41]:
pedlist=pd.concat([a,b,c])

In [42]:
pedlist.count()

牛号    7283320
dtype: int64

In [43]:
pedlist1=pedlist.drop_duplicates(keep='first').dropna(how='any')

In [44]:
pedlist1.count()

牛号    4023122
dtype: int64

In [45]:
len(pedlist1)

4023122

In [46]:
pedlist1['CODE']=range(1,len(pedlist1)+1)

In [47]:
pedlist1.count()

牛号      4023122
CODE    4023122
dtype: int64

# 匹配原牛号

In [48]:
pedlist2=pedlist1[:].rename(columns={'牛号':'母号','CODE':'DAM'})
pedlist3=pedlist1[:].rename(columns={'牛号':'父号','CODE':'SIRE'})

In [49]:
ped1.count()

牛号      2860944
出生日期    2788162
父号      2251424
母号      2170952
dtype: int64

In [50]:
ped2=pd.merge(ped1,pedlist1,on='牛号',how='left')
ped3=pd.merge(ped2,pedlist2,on='母号',how='left')
ped4=pd.merge(ped3,pedlist3,on='父号',how='left')

In [51]:
ped4.count()

牛号      2860944
出生日期    2788162
父号      2251424
母号      2170952
CODE    2860944
DAM     2170952
SIRE    2251424
dtype: int64

# 父母号唯一性验证

In [52]:
ped4['valid']=ped4['CODE']==ped4['SIRE']

In [53]:
ped4['valid1']=ped4['DAM']==ped4['SIRE']

In [54]:
ped4['valid2']=ped4['CODE']==ped4['DAM']

In [57]:
ped4['valid'].value_counts()

False    2860943
True           1
Name: valid, dtype: int64

In [58]:
ped4['valid1'].value_counts()

False    2856857
True        4087
Name: valid1, dtype: int64

In [59]:
ped4['valid2'].value_counts()

False    2859939
True        1005
Name: valid2, dtype: int64

一些牛的父号和母号是相同的，将这些数据都换成0。

In [63]:
ped4.loc[ped4.valid == True, 'SIRE'] = 0
ped4.loc[ped4.valid1 == True, 'SIRE'] = 0
ped4.loc[ped4.valid1 == True, 'DAM'] = 0
ped4.loc[ped4.valid2 == True, 'DAM'] = 0

In [66]:
ped5=ped4[['牛号','出生日期','父号','母号','CODE','DAM','SIRE']]

In [67]:
ped5.count() 

牛号      2860944
出生日期    2788162
父号      2251424
母号      2170952
CODE    2860944
DAM     2170952
SIRE    2251424
dtype: int64

In [68]:
ped5.to_csv("F:\\Python\\PED.csv",sep=",",encoding='utf_8_sig',index=False)